## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
np.random.seed(42)
torch.cuda.set_device(0)
import glob
import seaborn as sns

The first step is to partition the data into its categories: train/val/test

## Creating the dataframes

In [ ]:
#Categories list to DF
categories = pd.read_csv('./Anno/list_category_cloth.txt', sep='\s{2,}', header=0, skiprows=[0], engine='python')
categories['category_label'] = np.arange(1, len(categories)+1)

#DF with partition labels train/val/test
#DF with category number labels
img_partitions = pd.read_csv('./Eval/list_eval_partition.txt', delim_whitespace=True, header=1)
img_categories = pd.read_csv('./Anno/list_category_img.txt', delim_whitespace=True, header=0, skiprows=[0])

#Merge category DF with category label DF 
#Merge DFs for total DF with category and partition labels
img_categories = pd.merge(img_categories, categories, how='left', on='category_label' )
total_df = pd.merge(img_categories, img_partitions, how='left', on='image_name')

total_df now formulated, consists of the dataset with the evaluation status, cat type and cat name in one format

In [ ]:
print(total_df)

## EDA and Cleaning Data

manually cluster the similar categories from observations of the dataset

In [ ]:
#Combining "Caftan" and "Kaftan" (two different spellings) into one category
for i in total_df[(total_df['category_name'] == 'Caftan')].index:
    total_df.loc[i,'category_label'] = '43'
    total_df.loc[i,'category_name'] = 'Kaftan'
    

In [ ]:
def count_attr_imgs(attr_df):
    col_counts = {}

    for col in attr_df.columns[1:]:
        col_counts[col] = attr_df[col].value_counts().to_frame().loc[True][0]
    
    col_df = pd.DataFrame.from_dict(col_counts, orient='index', columns=['Count'])
    
    return col_df


In [ ]:
len(total_df[(total_df['category_name'] == 'Sarong')])

In [ ]:
drop_categories = total_df['category_name'].value_counts().tail(12)
print(drop_categories)

Now we want to clean the dataframe through dropping the categories that are attributes to the lowest populated classes

In [ ]:
cleaned_df = total_df.drop(total_df[total_df['category_name'].isin(drop_categories)].index).copy()

Now to see what the spread of data is within the dataset:

In [ ]:
cleaned_df['category_name'].value_counts().plot()

there is a high inbalance toward tshirts and dresses which may skew the model training - thus some of the highly populated categories are to be dropped

In [ ]:
arr2 = [1,3,4,234,2,35,24,5,345,34,23,87]
dropstuff = np.random.choice(arr2, 12)
print(dropstuff)

In [ ]:
def drop_cat(category, dropNum):
    cat_index = total_df[(total_df['category_name'] == category)&
                         (total_df['evaluation_status']
                           .isin(['train','val']))].index.tolist()
    print(cat_index)
    dropCat = np.random.choice(cat_index, dropNum, replace=False)
    return dropCat.tolist()

In [ ]:
drop_dress = drop_cat('Dress', 53000)
drop_tee = drop_cat('Tee', 22000)
drop_blouse = drop_cat('Blouse', 10000)
drop_shorts = drop_cat('Shorts', 6000)
drop_tank = drop_cat('Tank', 3000)
drop_skirt = drop_cat('Skirt', 2000)



drop_all_cat = drop_dress + drop_tee + drop_blouse + drop_shorts + drop_tank + drop_skirt
# print(drop_all_cat)

create training set with the categories dropped in place

In [ ]:
training_df = cleaned_df[cleaned_df['evaluation_status'].isin(['train','val'])][['image_name','category_name']].copy()
training_df.drop(drop_all_cat, inplace=True)
len(training_df)

split into test set below:

In [ ]:
test_df = cleaned_df[cleaned_df['evaluation_status'].isin(['test'])][['image_name','category_name']].copy()
len(test_df)
# test_df.drop(drop_all_cat, inplace=True)

In [ ]:
training_df['category_name'].value_counts()

In [ ]:
print(test_df)
print(training_df)

In [ ]:
test_df['category_name'].value_counts().plot()

In [ ]:
from matplotlib import image

# load the image
data = image.imread('Img/img/2-in-1_Space_Dye_Athletic_Tank/img_00000002.jpg')

plt.imshow(data)
plt.show()

## Model Training

In [ ]:
from fastai import *
from fastai.vision import *

np.random.seed(42)
torch.cuda.set_device(0)

### Loading the data in with DataBunch

In [ ]:
img_base_path = Path("Img/")
data = ImageDataBunch.from_df(img_base_path, training_df, ds_tfms=get_transforms(), size=224)
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(14,12))

Classes that exist within the data

In [ ]:
print(data.classes)
len(data.classes),data.c


### Different cnn_learners defined - utilised during the network analysis section

dropout of 0.2 and weight decay of 0.1  
use of resnet50 as final model

In [ ]:
# learn = cnn_learner(data, models.resnet50, metrics=accuracy)
# learn2 = cnn_learner(data, models.resnet34, metrics=accuracy)
# learn3 = cnn_learner(data, models.vgg19_bn, metrics=accuracy)


learn = cnn_learner(data, models.resnet50, metrics=accuracy, ps=0.2, wd=0.1)
                     

## Defining cross validation data split

make use off the sklearn library, using the StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

Create 5 fold 

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)


In [ ]:
def make_cnn_learner(data_fold):
    return cnn_learner(data_fold, models.resnet50, metrics=accuracy, ps=0.2, wd=0.1)

## 5 fold - define fold in each loop, 

In [ ]:
acc_val = []

for train_index, val_index in skf.split(training_df.index, training_df['category_name']):
    data_fold = (ImageList.from_df(training_df, img_base_path)
                 .split_by_idxs(train_index, val_index)
                 .label_from_df()
                 .transform(get_transforms(), size=224)
                 .databunch(num_workers=15)).normalize(imagenet_stats)
    learn = make_cnn_learner(data_fold)
    learn.fit_one_cycle(4)
    loss, acc = learn.validate()
    acc_val.append(acc.numpy())

initial top1 accuracy

In [ ]:
np.mean(acc_val)

In [ ]:
learn.save('cat-resnet50')
learn.export('cat-resnet50.pkl')

In [ ]:
learn4.fit_one_cycle(1)

In [ ]:
learn.load('cat-resnet50')

define top k prediction function

In [ ]:
def top_k_accuracy(output, target, topK=(3,)):
    maxK = max(topK)
    batch_size = target.size(0)
 
    _, prediction = output.topK(maxK, 1, True, True)
    prediction = prediction.t()
    correct = prediction.eq(target.view(1, -1).expand_as(prediction))
 
    result = []
    for k in topK:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        result.append(correct_k.mul_(100.0 / batch_size))
    return result

Top 5 from initial epoch:

In [ ]:
output, target = learn.get_preds()
top_k_accuracy(output, target)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('cat-resnet50-5epoch')

In [ ]:
learn.load('cat-resnet50-5epoch')

test top 3 again

In [ ]:
output, target = learn.get_preds()

top_k_accuracy(output, target, topk=(5,))

find optimal learner rate

In [ ]:
learn.lr_find()

Loop through the different learning rates to find lowest associate loss

In [ ]:
learn.recorder.plot()

unfreeze layers for training on entire network

In [ ]:
learn.unfreeze()

conduct another 8 epochs with max learning rate defined within slice according to lowest loss. Following the one cycle policy

In [ ]:
learn.fit_one_cycle(8, max_lr=slice(1e-5,1e-3))

In [ ]:
learn.export('cat-resnet50-5epoch-8uf.pkl')

In [ ]:
learn.save('cat-resnet50-5epoch-8uf')

freeze all layers except for the head layer

In [ ]:
learn.freeze()

train head layer another 4 epochs

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.export('cat-resnet50-9epoch-8uf.pkl')

In [ ]:
learn.save('cat-resnet50-9epoch-8uf')

In [ ]:
learn.load('cat-resnet50-9epoch-8uf')

check top-5 prediction

In [ ]:
output, target = learn.get_preds()
top_k_accuracy(output, target, topk=(5,))

top 5 prediction is outputting 2% better than the FashionNet research

top-3 prediction ()

In [ ]:
output, target = learn.get_preds()
top_k_accuracy(output, target)

Find highest lr associated to lowest loss again for a varying learning rate

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

unfreeze the layers once again to make them all trainable

In [ ]:
learn.unfreeze()

redefined the max_lr slice 

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-2))

In [ ]:
learn.export('cat-resnet50-9epoch-12uf.pkl')

In [ ]:
learn.save('cat-resnet50-9epoch-12uf.pkl')

In [ ]:
learn.load('cat-resnet50-9epoch-12uf.pkl')

Top 3

In [ ]:
output, target = learn.get_preds()
top_k_accuracy(output, target, topk=(3,))

Top 5

In [ ]:
output, target = learn.get_preds()
top_k_accuracy(output, target, topk=(5,))

### Observe the most confused categories 

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=80)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

#### Test predictions on validation set for top-3 top-5

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Valid)
top_k_accuracy(output, target, topk=(3,))

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Valid)
top_k_accuracy(output, target, topk=(5,))

In [ ]:
def top_k_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

#### Test predictions on train set for top 3, top5, expect higher fitting

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Train)
top_k_accuracy(output, target, topk=(3,))

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Train)
top_k_accuracy(output, target, topk=(5,))

### Utilising test set

In [ ]:
img_base_path = Path("Img/")
testdata = ImageDataBunch.from_df(img_base_path, test_df, ds_tfms=get_transforms(), size=224)
testdata.normalize(imagenet_stats)

In [ ]:
learn.data = testdata

In [ ]:
print(learn.data)

Evaluating based upon the Test dataset:  
    top3:

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Train)
top_k_accuracy(output, target, topk=(3,))

top5:

In [ ]:
output, target = learn.get_preds(ds_type=DatasetType.Train)
top_k_accuracy(output, target, topk=(5,))

## Real world example 

In [ ]:
predict_img_path = "./data/zen5.jpg"
show_image(open_image(predict_img_path))
learn = load_learner(path='./Img/', file='cat-rn50-size150-fr4-unfr2-fr4-unfr4.pkl')
classes = ['Blazer', 'Blouse', 'Bomber', 'Button-Down', 'Cardigan', 'Chinos', 'Coat', 'Culottes', 'Cutoffs', 'Dress', 'Flannel', 'Henley', 'Hoodie', 'Jacket', 'Jeans', 'Jeggings', 'Jersey', 'Joggers', 'Jumpsuit', 'Kimono', 'Leggings', 'Parka', 'Poncho', 'Romper', 'Shorts', 'Skirt', 'Sweater', 'Sweatpants', 'Sweatshorts', 'Tank', 'Tee', 'Top', 'Trunks']


In [ ]:
category,classIndex,losses = learn.predict(open_image(predict_img_path))
predictions = sorted(zip(classes, map(float, losses)), key=lambda p: p[1], reverse=True)
print (predictions[:5])
print (len(predictions))

### Mocking of server based communication

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
import requests

In [ ]:
url='http://localhost:8889/api'
data = json.dumps({
    'base64':'...'
})
r = requests.post(url, data)
print(r.text)

In [ ]:
import dill

In [ ]:
dill.dump_session('cat_notebook_env.db')